In [1]:
import math

In [3]:
def fast_exp(base, exponent, modulus):
  result = 1
  while exponent != 0:
    if exponent % 2 == 1:
      result = (result * base) % modulus
    base = base**2 % modulus
    exponent = exponent // 2
  return result

def extended_euclidean(m, n):
  if n == 0:
    return 1, 0, m
  x, y, g = extended_euclidean(n, m % n)
  return y, x - (m // n)*y, g

def inv_mod(a, modulus):
  s, _, g = extended_euclidean(a, modulus)
  assert g == 1, ValueError('the modular inverse does not exist')
  return s % modulus

def crt(remainders, moduli):
  assert len(remainders) == len(moduli), ValueError('the lists remainders and moduli are not the same length')
  assert len(remainders) > 0, ValueError('the list lengths must be greater than one')

  first_modulus = moduli[0]
  first_remainder = remainders[0]

  if len(remainders) == 1:
    return first_remainder % first_modulus, first_modulus
  
  consecutive_remainder, consecutive_modulus = crt(remainders[1:], moduli[1:])
  u, v, g = extended_euclidean(consecutive_modulus, first_modulus)

  assert g == 1, ValueError('the moduli are not relatively prime')

  return (first_remainder*consecutive_modulus*u + consecutive_remainder*first_modulus*v) % (first_modulus*consecutive_modulus), first_modulus*consecutive_modulus

def polynomial_root(exponent, a, modulus):
  u = inv_mod(exponent, modulus - 1)

  result = fast_exp(a, u, modulus)

  return result

def cf_float(x, n):
  cf = []
  for _ in range(n):
    cf.append(int(x))
    x = 1 / (x % 1)
  return cf

def continued_frac(num, den):
  cf = []
  while den != 0:
    cf.append(num//den)
    num, den = den, num % den
  return cf

def get_convergents(cf):
  convergents = [[0, 1], [1, 0]]
  for a in cf:
    hk2, hk1 = convergents[-2:]
    convergents.append([a*hk1[0] + hk2[0], a*hk1[1] + hk2[1]])
  return convergents[2:]

def crack_rsa(e, n):
  convergents = get_convergents(continued_frac(e, n))
  for convergent in convergents:
    k, d = convergent
    if k != 0 and (phi := (d * e - 1) // k) != 0:
      p_q = n - phi + 1
      disc = p_q**2 - 4*n
      if disc > 0:
        p, q = (p_q + math.isqrt(disc)) // 2, (p_q - math.isqrt(disc)) // 2
        if p*q == n:
          return d, p, q
  return -1, -1, -1

def pollard_rho(n):
  f = lambda x_: x_**2 + 1
  x = 2
  y = 2
  g = 1
  count = 0
  while g == 1:
    x = f(x) % n
    y = f(f(y)) % n
    _, _, g = extended_euclidean(abs(x-y), n)
    count += 1
  return g, count

def fast_exp_mont(base, k, modulus):
  R_0 = 1
  R_1 = base
  exponent = format(k, 'b')
  for bit in exponent:
    if bit == '0':
      R_1 = R_1 * R_0 % modulus
      R_0 = R_0**2 % modulus
    else: 
      R_0 = R_1 * R_0 % modulus
      R_1 = R_1**2 % modulus
  return R_0

In [ ]:
class EC_Point:
  def __init__(self, x, y, a, b, modulus, zero=False):
    assert (y**2 - (x**3 + a*x + b)) % modulus == 0 or zero, ValueError(f'({x}, {y}) does not satisfy y**2 = x**3 + {a}*x + {b} (mod {modulus})')
    
    self.x = x
    self.y = y
    self.a = a
    self.b = b
    self.modulus = modulus
    self.zero = zero

  def __str__(self):
    if self.zero:
      return '0'
    
    return f'({self.x}, {self.y})'

  def __repr__(self):
    return f'{self.__class__.__name__}(x={self.x}, y={self.y}, a={self.a}, b={self.b}, modulus={self.modulus}, zero={self.zero})'

  def __add__(self, Q):
    assert self.a == Q.a, ValueError('a values do not match')
    assert self.b == Q.b, ValueError('b values do not match')
    assert self.modulus == Q.modulus, ValueError('moduli do not match')

    if self.zero:
      return EC_Point(Q.x, Q.y, Q.a, Q.b, Q.modulus, Q.zero)

    if Q.zero:
      return EC_Point(self.x, self.y, self.a, self.b, self.modulus, self.zero)

    s = 0
    
    if (self.x - Q.x) % self.modulus != 0:
      s = (Q.y - self.y)*inv_mod(Q.x - self.x, self.modulus) % self.modulus
    
    elif (self.y - Q.y) % self.modulus != 0:
      return EC_Point(self.x, self.y, self.a, self.b, self.modulus, zero=True)

    else:
      if self.y == 0:
        return EC_Point(self.x, self.y, self.a, self.b, self.modulus, zero=True)

      else:
        s = (3*self.x**2 + self.a)*inv_mod(2*self.y, self.modulus) % self.modulus

    x_r = (s**2 - self.x - Q.x) % self.modulus
    y_r = -(self.y + s*(x_r - self.x)) % self.modulus
    return EC_Point(x_r, y_r, self.a, self.b, self.modulus)

  def __sub__(self, Q):
    return self + EC_Point(Q.x, -Q.y % Q.modulus, Q.a, Q.b, Q.modulus, Q.zero)

  def __mul__(self, k):
    assert k >= 0, ValueError('cannot use a negative scalar')

    total = EC_Point(self.x, self.y, self.a, self.b, self.modulus, zero=True)

    if not self.zero:
      multiple = EC_Point(self.x, self.y, self.a, self.b, self.modulus, self.zero)
      while k != 0:
        if k % 2 == 1:
          total = total + multiple
        k = k // 2
        multiple = multiple + multiple
    return total

  __rmul__ = __mul__